## kepp-alive优化网页性能

每一次路由切换,ajax都会重新发送


![](https://upload-images.jianshu.io/upload_images/7505161-3345805c54d167c0.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)




`<router-view/>`显示路由对应的内容.

```js
<template>
  <div id="app">
    <!--显示路由对应的内容-->
    <keep-alive>
      <router-view/>
    </keep-alive>
  </div>
</template>
```

`<keep-alive/>`是vue自带的标签.把路由加载一次之后,就把路由中的内容放到内存当中.

下一次再进入路由当中,不需要再重新渲染组件.

## 携带store.js中的state的数据进行传递,达到切换不同城市换不同城市页面的目的.

```js
import { mapState } from 'vuex'

axios.get('api/data?city=' + this.city){}
    
computed: {
...mapState(['city'])
},

```

## 改变缓存中的内容(actived)

利用生命周期函数`actived()`函数.我们从首页的组件进入到City组件,会触发`mounted & actived`组件,再从City组件,回到首页组件,只会触发`actived`组件.

所以`actived`是页面重新被显示的时候被执行.

**target**: 
- 第二次选择相同城市的时候,我们不希望再发送一次ajax请求.
- 第二次选择不同城市的时候,我们发送新的ajax请求.

    
```js
 mounted () {
    this.lastCity = this.city
    this.getHomeInfo()
  },
  activated () {
    // 判断之前的城市和当前显示的城市是否相同
    // 不相同,再放一次ajax请求
    if (this.lastCity !== this.city) {
      this.lastCity = this.city
      this.getHomeInfo()
    }
  }
}

```

## 动态路由的使用

```js
router-link.item.border-bottom(v-for="item of list" :keys="item.id" tag="li" :to="'./detail/'+ item.id")
```
```js
{
      path: '/detail/:id',
      name: 'detail',
      component: Detail
}
```

## 创建公用组件

在`SRC`目录下创建一个`common/gallary`的公用组件来进行测试.

```
overflow: hidden会将高度限制死,导致别处的高度无法扩展.
```
## Bug

遇到一个bug,在使用`vue-awesome-swiper`的时候,我需要将显示的当前页数,往下面调,而不是遮挡住图片.

由于是`absolute`定位,所以我就调节bottom,但是却不显示.我猜测是`overflow:hidden`搞得鬼,所以我找了这个容器的第一个父级容器,将其设置为`inherit`,还是不行.

然后我再去找他的父级容器,再将`oveflow设置为inherit`

## activated钩子


- activated会在每次页面展示的时候执行.
- deactivated会在页面即将被替换的时候执行.

- activated 生命周期在keep-alive 组件激活时调用
- 服务器端渲染期间不调用该生命周期
- 通常和deactivated周期一起使用

## 手机屏幕Scroll(拖动)的函数

`window.addEventListener('scroll', this.handleScroll)`

## 给组件添加样式

`:style="opacityStyle"`

```js
methods: {
    handleScroll () {
      const top = document.documentElement.scrollTop
      if (top > 60) {
        let opacity = top / 140
        opacity = opacity > 1 ? 1 : opacity
        this.opacityStyle = { opacity }
        this.showAbs = false
      } else {
        this.showAbs = true
      }
    }
  },
```

## 对全局事件的解绑

`window.addEventListener('scroll', this.handleScroll)`

这段话绑定在了`window`这个全局对象中.所以我们需要对全局事件进行解绑.

所以我们需要在每次页面即将被替换的时候移除`scroll`事件.

```js
deactivated(){
 window.remove('scroll',this.handleScroll)   
}
```


## 递归组件的使用

```json
    list: [{
        title: '成人票',
        children:[{
            title: '成人三馆联票'
        },{
            title: '成人五馆联票'
        }]
      }, {
        title: '学生票'
      }, {
        title: '儿童票'
      }, {
        title: '特惠票'
      }]
    }
```

我们希望在页面上显示出一级标题以后,也同时显示二级标题.

递归组件: 使用组件自身调用自身.

In [7]:
def fact(n):
    if n==1:
        return 1
    return n * fact(n - 1)
fact(3)

#
#
# fact(3) = 3 * f(2)
# fact(2) = 2 * f(1)
# f(1) = 1


6

```js
.list-container
    .item-title.border-bottom(v-for="item in list" :key="item.id")
      span.item-title-icon
      e {{item.title}}
      .item-children(v-if="item.children")
        detail-list(:list="item.children")
```

![](https://upload-images.jianshu.io/upload_images/7505161-4984efdcbfe981ee.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)


## 动态的获取Ajax的请求

```
  methods: {
    getDetailInfo () {
      axios.get('api/detail?id=' + this.$route.params.id)
        .then(this.getDetailInfoSuccess)
    },
    getDetailInfoSuccess (res) {
      console.log(res)
    }
  },
  mounted () {
    this.getDetailInfo()
  }
```

## ajax动态数据的获取的改写方式.
```js
  methods: {
    getDetailInfo () {
      axios.get('api/detail', {
        // there replace above write method
        params: {
          id: this.$route.params.id
        }
      }).then(this.getDetailInfoSuccess)
    },
    getDetailInfoSuccess (res) {
      console.log(res)
      res = res.data
      if (res.ret && res.data) {
        const data = res.data
        this.bannerImg = data.bannerImg
        this.categoryList = data.categoryList
        this.gallaryImgs = data.gallaryImgs
        this.sightName = data.sightName
      }
    }
  },
  mounted () {
    this.getDetailInfo()
  }
}
```

## 特定页面不做缓存

```js
<keep-alive exclude="Detail">
      <router-view/>
</keep-alive>
```

剔除`Detail`这个页面不做缓存,也就是每次重新进入页面,重新发一次ajax请求.

## 每个页面的name有什么作用?

- 做递归缓存的时候,用到了
- 取消页面缓存的时候,用到了
- Vue的调试工具,选中组件的时候会显示组件名称

## 动画效果的添加

**FadeAnimation.vue**

```js
<template lang="pug">
  transition
    slot

</template>
<script type="text/ecmascript-6">
export default {
  name: 'Fade'
}
</script>
<style lang="postcss" scoped>
.v-enter,.v-leave-to{
  opacity: 0;
}
.v-enter-active,.v-leave-active{
 transition: opacity .5s;
}

</style>

```

使用方式

```pug
fade
 common-gallary(:imgs="gallaryImgs" v-show="showGallary" @close="handleCloseGallary")
```

此刻,`common-gallary`就以插槽的形式在`fade`组件里面显示.也就是说,上面的`slot`就是现在的`common-gallary`

## 阻止touchStart事件的默认行为

```js
@touchstart.prevent="handleTouchStart"
```

## 手机访问网页是白屏的效果的解决方式
原因:

- 手机浏览器默认不支持`Promise`

安装`babel-polyfill`,如果手机不支持Es6的新特性,`polyfill`会自动添加这些新特性.


## Vue知识学习

vue-router
vuex
服务器端渲染
vue插件
vue源码

In [ ]:
365 - 175.5 - 175.5 = 14(剩余空间)


